In [11]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase
import os
import langchain_core

In [2]:
from Flemmings_Library.variables import db_path, db
from Flemmings_Library.ollama import get_ollama_model_names

!ollama list > nul 2>&1 # Für Windows: Ollama starten, Ausgabe der Modelle unterdrücken

In [3]:
model_names = get_ollama_model_names(debug=False)
db_dialect = db.dialect
model_name = model_names[0] # Modell festlegen

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
)
print(f'{model_name} ausgewählt und geladen.')

llama3.2-vision:11b ausgewählt und geladen.


Schema extrahieren

In [5]:
from Flemmings_Library.extract import extract_schema

schema = extract_schema(db_path, sample_rows=1, verbose=False)
schema_info_str = str(schema)

Prompt erstellen

In [6]:
from Flemmings_Library.variables import prompt_template_sql

In [7]:
question = "In which city is the customer with the highest customer ID located?"

In [8]:
prompt = prompt_template_sql.format(
    schema_info=schema_info_str, 
    question=question, 
    db_dialect=db_dialect
    )

In [13]:
prompt.pretty_print()

AttributeError: 'str' object has no attribute 'pretty_print'

In [14]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

# Erstelle das PromptTemplate
prompt_template = ChatPromptTemplate(
    messages=[
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template("{query}")
    ]
)

# Zeige die schön formatierte Darstellung der Prompt-Vorlage
prompt_template.pretty_print()

# Formatierung liefert einen String – diesen kannst du dann normal mit print() ausgeben
formatted_prompt = prompt_template.format(chat_history=[], query="Wie viele Einträge gibt es?")
print(formatted_prompt)


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{query}
Human: Wie viele Einträge gibt es?


Ab hier mit ChatGPT

In [ ]:
from langchain.chains import LLMChain
from typing_extensions import TypedDict
import re

# Define your State class
class State(TypedDict):
    schema_info: str       # Loaded information about the DB schema
    prompt: str
    question: str          # Original natural language question
    db_dialect: str        # SQL dialect (e.g., "sqlite")
    parsed_question: str   # (Optional) Parsed or analyzed question
    query: str             # Generated SQL query
    execution_result: str  # Result from executing the SQL query
    answer: str            # Final answer formatted for the user
    error: str             # Error message (if any)

In [ ]:
# Legacy
# chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm | StrOutputParser()

In [ ]:
# initialen State festlegen
state: State = {
    "schema_info": schema_info_str,
    "question": question,
    "db_dialect": db.dialect,
    "prompt": prompt,
    "parsed_question": "",     
    "query": "",
    "execution_result": "",
    "answer": "",
    "error": ""
}

In [ ]:
output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })

In [ ]:
# Update für den State
state['answer'] = output

In [ ]:
state

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe

In [ ]:
output

'SELECT City FROM Customer WHERE CustomerId = ( SELECT MAX(CustomerId) FROM Customer )'

# Old Stuff

In [ ]:
# New helper function that uses chain.invoke and updates the State
def extract_sql_query_state(prompt_template, state: State) -> State:
    """
    Uses an LLMChain to generate an SQL query from the natural language question 
    contained in the provided state. Updates the state's 'query' field with the 
    extracted SQL and sets an 'error' field if exceptions occur.
    """
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    try:
        # Call chain.invoke with a dictionary as input
        output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })
        
        # Optionally extract the SQL statement (if extra formatting is added)
        sql_query = extract_sql_statement(output)
        state["query"] = sql_query if sql_query else output
        state["error"] = ""
    except Exception as e:
        state["error"] = str(e)
    
    return state

# Example usage:

# Assuming you have created the prompt_template and have the following variables defined:
# schema_info_str, question, and db_dialect


In [ ]:
# Invoke the chain to update the state with the generated SQL query
state = extract_sql_query_state(prompt_template, state)

print("Generated SQL Query:")
print(state["query"])
if state["error"]:
    print("Error encountered:", state["error"])


C:\Users\flemm\AppData\Local\Temp\ipykernel_11580\2016002811.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


Generated SQL Query:

Error encountered: name 'extract_sql_statement' is not defined


In [ ]:
# Wenn output ein Dictionary ist und den Schlüssel "text" enthält, verwende dessen Inhalt
if isinstance(output, dict) and "text" in output:
    sql_query = extract_sql_statement(output["text"])
else:
    sql_query = extract_sql_statement(output)


In [ ]:
def extract_sql_query_state(prompt_template, state: State) -> State:
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    try:
        output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })
        
        # Extrahiere den SQL-Text aus dem Dictionary, falls vorhanden
        if isinstance(output, dict) and "text" in output:
            sql_query = extract_sql_statement(output["text"])
        else:
            sql_query = extract_sql_statement(output)
        
        state["query"] = sql_query if sql_query else output
        state["error"] = ""
    except Exception as e:
        state["error"] = str(e)
    
    return state


In [ ]:
extract_sql_query_state(prompt_template, state)

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe